## Aufgabe
1) Logistische Regression - Spamklassifikation
    - Führen Sie die Spamklassifikation der vorherigen Übung mit einer logistischen Regression durch.
    - Erstellen Sie für die Auswertung eine ROC-Kurve.
2) Klassifikation - SA-Subset
    - Selektieren Sie nur Reihen mit den 3 häufigsten Labels aus dem SA-Subset (df = df[df['labels'].isin([b'normal.', b'smurf.', b'neptune.'])].copy()).
    - Führen Sie eine Klassifikation dieser 3 Klassen durch und finden Sie den besten Parameter für min_freq.

### Notizen

#### Linear Regression
1) Lineare Kombination der Features
    - $z = w_{1}x_{1} + w_{2}x_{2} + \ldots + w_{n}x_{n} + b$
        - $w_{i}$: Gewichte
        - $x_{i}$: Featurewerte
        - $b$: Bias (auch $-\Theta$)
2) Anwendung der Sigmoid Funktion
    - $\hat{y}=\sigma(z)=\frac{1}{1+\mathscr{e}^{-z}}$
    - $\hat{y}$ ist die vorhergesagte wahrscheinlichkeit, dass das Sample 1 ist

#### Loss Function
...

### Aufgaben

#### Imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Aufgabe 1

##### Dataset

In [6]:
data = pd.read_csv(
    'https://raw.githubusercontent.com/AiDevNepal/ai-saturdays-workshop-8/master/data/spam.csv'
)
data

,text,target
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam
5568,Will Ì_ b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


In [8]:
data['target'].value_counts()

target
ham     4825
spam     747
Name: count, dtype: int64

In [9]:
data.describe()

,text,target
count,5572,5572
unique,5169,2
top,"Sorry, I'll call later",ham
freq,30,4825


In [10]:
data.dtypes

text      object
target    object
dtype: object